# Dependencies

install `StormFront` either as a cluster library using `git+https://www.github.com/PavanDendi/StormFront` as the library name, or as a notebook scoped library using `%pip install git+https://www.github.com/PavanDendi/StormFront`

Install `sqlstorm` jar from https://github.com/PavanDendi/sqlstorm to your cluster.

Compiled binary can be found in the github Releases [HERE](https://github.com/PavanDendi/sqlstorm/raw/main/bin/sqlstorm.jar), or compile from source using `sbt assembly` from the sql⚡torm repo root directory.

Cluster can be a single node using a weak instance type since the cluster will only be sending API calls, and processing minimal data itself.

Cluster must be running a DBR version that supports ipykernel. Tested on DBR 10.4 LTS

If using DBR 11+, ipykernel is enabled by default.
For 11 > DBR >= 10.3, enable ipykernel by adding to cluster config
```
spark.databricks.python.defaultPythonRepl ipykernel
```

In [ ]:
# Select SQL Files to Run

from StormFront import app

# display(app.start("/Users/pavan.dendi@databricks.com/"))
display(app.start())

In [ ]:
# Select DB SQL Endpoint

from StormFront.db_select import SQLEPConfig

# PAT token needed in order to query the workspace for DB SQL Endpoints
# See https://docs.databricks.com/dev-tools/api/latest/authentication.html
#    for instructions on how to generate a PAT
# See https://docs.databricks.com/security/secrets/index.html
#    for how to use the Secrets feature to securely store credentials
token = dbutils.secrets.get("fieldeng", "prd-pat")
ep_sel = SQLEPConfig(token)
display(ep_sel)

In [ ]:
# Test JDBC Connection and sql⚡torm Config

from pathlib import PosixPath, Path
from StormFront import app
from StormFront.utils import *

jdbc = ep_sel.get_jdbc()
dbstress_cfg = DBstressCfg(
    yaml_path="/dbfs/tmp/dbstress/config.yaml",
    result_path="/dbfs/tmp/dbstress/output/"
)
conn_cfg = ConnConfig(
    parallel=1,
    repeats=1,
    timeout=3000
)

display(app.run(jdbc, [Query("test", "SELECT 1")], dbs_cfg=dbstress_cfg, conn=conn_cfg))

In [ ]:
# Run dbstress with Selected SQL files and JDBC Connection

from pathlib import Path
from StormFront.utils import read_sql_files, ConnConfig, DBstressCfg


jdbc = ep_sel.get_jdbc()
queries = read_sql_files(app.get_selected(), info=True)

dbstress_cfg = DBstressCfg(
    yaml_path="/dbfs/tmp/dbstress/config.yaml",
    result_path="/dbfs/tmp/dbstress/output/"
)

conn_cfg = ConnConfig(
    parallel=1,
    repeats=1,
    timeout=3000
)

display(app.run(jdbc, queries, dbs_cfg=dbstress_cfg, conn=conn_cfg))